In [368]:
import sys
import requests
import pandas as pd

In [369]:
# url = https://data.census.gov/mdat/#/search?ds=CPSBASIC202007&cv=PESEX,PRMJIND1,PRMJOCC1&rv=PEMLR,PTDTRACE&wt=PWCMPWGT

keys = ENTER YOUR KEY

#url = 'https://api.census.gov/data/2020/cps/basic/jul?get=PWCMPWGT,PRMJIND1,PRMJOCC1,PEMLR,PESEX,PTDTRACE'

In [372]:
def get_cps(year,month):
    
    url = 'https://api.census.gov/data'
    url = url + '/' + year + '/cps/basic/' + month + '?tabulate=weight(PWCMPWGT)'
    url = url + '&col+PRMJIND1&col+PRMJOCC1&col+PEMLR'
    url = url + '&row+PESEX&row+PTDTRACE'
    
    url = url + '&key=' + keys
    
    # Request data
    r = requests.get(url).json()
    
    rownum = 2
    temp0 = [list(r[0][i].values()) for i in range(len(r[0])-rownum)]
    temp0 = pd.DataFrame(temp0,dtype='float64')
    temp0.columns = ['PRMJIND1','PRMJOCC1','PEMLR']
    
    temp1 = pd.DataFrame(r[1:])
    temp1 = temp1.astype(int)
    
    df = pd.DataFrame(columns=['PRMJIND1','PRMJOCC1','PEMLR','DATA','SEX','RACE'])
    
    for i in range(temp1.shape[0]):
        tempT = temp1.iloc[:,:-rownum].T.iloc[:,i]
        tempT = pd.DataFrame(tempT.values,columns=['DATA'])
        tempT.loc[:,'SEX'] = temp1.iloc[i,-rownum]
        tempT.loc[:,'RACE'] = temp1.iloc[i,-rownum+1]

        tempX = pd.concat([temp0,tempT],axis=1)

        df = pd.concat([df,tempX],axis=0)

    df = df.query('PRMJIND1 > 0 & PRMJOCC1 > 0 & (PEMLR == 1 | PEMLR == 2)')
    df = df.sort_values(['PRMJOCC1','SEX','RACE','PEMLR','PRMJIND1'])
    df = df.reindex(columns=['PRMJOCC1','SEX','RACE','PRMJIND1','PEMLR','DATA'])

    df = df.fillna(0).astype(int)

    temp00 = df[df['PEMLR']==1.0]
    temp00 = temp00.reset_index(drop=True)

    temp10 = df[df['PEMLR']==2.0]
    temp10 = temp10.reset_index(drop=True)

    temp00['DATA_ALL'] = temp00['DATA'] + temp10['DATA']

    temp00 = temp00.drop(['DATA','PEMLR'],axis=1)
    
    colname = ['PRMJOCC1','SEX','RACE',
           '1','2','3','4','5','6','7','8','9','10','11','12','13','14']
    
    df0 = pd.DataFrame(columns=colname)
    
    for i in range(int(temp00.shape[0]/14)):
        added = list(temp00.iloc[i*14,:3].values) + list(temp00.iloc[i*14:(i+1)*14,4])
        added = pd.DataFrame(added,index=colname).T
        df0 = pd.concat([df0,added],axis=0)

    df0 = df0.set_index(['PRMJOCC1','SEX','RACE'])
    
    df0.columns = ['Agriculture etc',
              'Mining',
              'Construction',
              'Manufacturing',
              'Wholesale and retail trade',
              'Transportation and utilities',
              'Information',
              'Financial activities',
              'Professional and business services',
              'Educational and health services',
              'Leisure and hospitality',
              'Other services',
              'Public administration',
               'Armed Forces']

    return df0

In [373]:
df = get_cps('2020','jul')
df

Agriculture etc  Mining Construction Manufacturing  \
PRMJOCC1 SEX RACE                                                      
1        1   1             683363  104209      1744343       1575507   
             2               1427       0        69556         98812   
             3               8535       0         8549          7010   
             4               1021       0        31414         99823   
             5                  0       0         3549             0   
...                           ...     ...          ...           ...   
11       2   22                 0       0            0             0   
             23                 0       0            0             0   
             24                 0       0            0             0   
             25                 0       0            0             0   
             26                 0       0            0             0   

                  Wholesale and retail trade Transportation and utilities  \
PRMJOCC1 SEX RACE                                                           
1        1   1                        798816                       537937   
             2                         65898                        74051   
             3                          5339                            0   
             4                         66447                        56234   
             5                          2593                            0   
...                                      ...                          ...   
11       2   22                            0                            0   
             23                            0                            0   
             24                            0                            0   
             25                            0                            0   
             26                            0                            0   

                  Information Financial activities  \
PRMJOCC1 SEX RACE                                    
1        1   1         328854              2183086   
             2          26335               173634   
             3              0                13892   
             4          27958               179338   
             5              0                    0   
...                       ...                  ...   
11       2   22             0                    0   
             23             0                    0   
             24             0                    0   
             25             0                    0   
             26             0                    0   

                  Professional and business services  \
PRMJOCC1 SEX RACE                                      
1        1   1                               2333436   
             2                                185008   
             3                                  5322   
             4                                279110   
             5                                     0   
...                                              ...   
11       2   22                                    0   
             23                                    0   
             24                                    0   
             25                                    0   
             26                                    0   

                  Educational and health services Leisure and hospitality  \
PRMJOCC1 SEX RACE                                                           
1        1   1                             974081                  840368   
             2                             142514                  119412   
             3                              14873                       0   
             4                              71151                  115845   
             5                              11434                    1172   
...                                           ...                     ...   
11       2   22                

In [5]:
#PRMJIND1
#-1 
#1 Agriculture, forestry, fishing, and hunting
#2 Mining
#3 Construction
#4 Manufacturing
#5 Wholesale and retail trade
#6 Transportation and utilities
#7 Information
#8 Financial activities
#9 Professional and business services
#10 Educational and health services
#11 Leisure and hospitality
#12 Other services
#13 Public administration
#14 Armed Forces

#PRMJOCC1
#-1
#1	Management, business, and financial occupations
#2	Professional and related occupations
#3	Service occupations
#4	Sales and related occupations
#5	Office and administrative support occupations
#6	Farming, fishing, and forestry occupations
#7	Construction and extraction occupations
#8	Installation, maintenance, and repair occupations
#9	Production occupations
#10	Transportation and material moving occupations
#11	Armed Forces

#PEMLR
#-1
#1 Employed-At Work
#2 Employed-Absent
#3 Unemployed-On Layoff
#4 Unemployed-Looking
#5 Retired-Not In Labor Force
#6 Disabled-Not In Labor Force
#7 Other-Not In Labor Force

#PTDTRACE
#01	White Only
#02	Black Only
#03	American Indian, Alaskan Native Only
#04	Asian Only
#05	Hawaiian/Pacific Islander Only
#06	White-Black
#07	White-AI
#08	White-Asian
#09	White-HP
#10	Black-AI
#11	Black-Asian
#12	Black-HP
#13	AI-Asian
#14	AI-HP
#15	Asian-HP
#16	W-B-AI
#17	W-B-A
#18	W-B-HP
#19	W-AI-A
#20	W-AI-HP
#21	W-A-HP
#22	B-AI-A
#23	W-B-AI-A
#24	W-AI-A-HP
#25	Other 3 Race Combinations
#26	Other 4 and 5 Race Combinations